In [1]:
'''
In the messy dataset the following things have to be cleaned up: 
- missing values 
- random string values
- swapped columns 
- numeric distortion (numbers have been multiplied, scaled or randomly altered) 
- truncated text
'''
import pandas as pd

df = pd.read_csv("../../data/raw/dirty_detailed_listings_data.csv")

df

/var/folders/yw/8qjcs1g93ws0j0rgq361rm3h0000gn/T/ipykernel_2989/2023414818.py:11: DtypeWarning: Columns (30,37,42,51,67,75,78) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../../data/raw/dirty_detailed_listings_data.csv")


,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,13913,https://www.airbnb.com/rooms/13913,20250914034649,2025-09-16,city scrape,Holiday London DB Room Let-on going,My bright double bedroom with a large window h...,Finsbury Park is a friendly melting pot commun...,https://a0.muscache.com/pictures/miso/Hosting-...,54730,...,4.87,4.78,4.78,NaN,f,2,1.0,1,0,0.3
1,15400,https://www.airbnb.com/rooms/15400,20250914034649,2025-09-16,city scrape,Bright Chelsea Apartment. Chelsea!,Lots of windows and light. St Luke's Gardens ...,It is Chelsea.,https://a0.muscache.com/pictures/428392/462d26...,60302,...,4.84,4.93,4.74,NaN,f,1,1.0,0,0,0.51
2,17402,https://www.airbnb.com/rooms/17402,20250914034649,2025-09-16,city scrape,Very Central Modern 3-Bed/2 Bath By Oxford St W1,"You'll have a great time in this beautiful, cl...","Fitzrovia is a very desirable trendy, arty and...",https://a0.muscache.com/pictures/39d5309d-fba7...,67564,...,4.72,4.89,4.61,NaN,f,2,2.0,0,0,0.32
3,24328,https://www.airbnb.com/rooms/24328,20250914034649,2025-09-18,previous scrape,Battersea live/work artist house,"Artist house by SW Battersea Park, bright high...","- Battersea is a quiet family area, easy acces...",https://a0.muscache.com/pictures/9194b40f-c627...,41759,...,4.93,4.6,4.65,NaN,f,1,1.0,0,0,0.53
4,36274,https://www.airbnb.com/rooms/36274,20250914034649,2025-09-15,city scrape,Bright 1 bedroom apt off brick lane in Shoreditch,*Update June '25- Pump Installed to improve wa...,NaN,https://a0.muscache.com/pictures/hosting/Hosti...,133271,...,4.46,4.85,4.54,NaN,t,2,2.0,0,0,0.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96866,1508894090797273412,https://www.airbnb.com/rooms/1508894090797273412,20250914034649,2025-09-16,city scrape,"Blueground | Finsbury, balcony, nr St. Paul's",Feel at home wherever you choose to live with ...,NaN,https://a0.muscache.com/pictures/prohost-api/H...,314162972,...,NaN,NaN,NaN,NaN,t,405,405.0,0,0,NaN
96867,1508900042872179492,https://www.airbnb.com/rooms/1508900042872179492,20250914034649,2025-09-15,city scrape,Self-Contained Studio in Heart of Tooting Broa...,Welcome to your perfect London stay - The Lond...,NaN,https://a0.muscache.com/pictures/hosting/Hosti...,718690455,...,5.0,5.0,5.0,NaN,f,1,1.0,0,0,2.0
96868,1508926597927944565,https://www.airbnb.com/rooms/1508926597927944565,20250914034649,2025-09-14,city scrape,One bedroom apartment Dagenham,"Modern 1-bedroom apartment in Kerwin House, Da...",NaN,https://a0.muscache.com/pictures/hosting/Hosti...,389056540,...,NaN,NaN,NaN,NaN,f,1,1.0,0,0,NaN
96869,1508962439633147670,https://www.airbnb.com/rooms/1508962439633147670,20250914034649,2025-09-15,city scrape,Short Stay,Take it easy at this unique and tranquil getaway.,NaN,https://a0.muscache.com/pictures/hosting/Hosti...,683246718,...,NaN,NaN,NaN,NaN,f,1,0.0,1,0,NaN


In [2]:
df.isna().sum()

id                                                 18
listing_url                                        19
scrape_id                                          16
last_scraped                                       27
source                                             26
                                                ...  
calculated_host_listings_count                     25
calculated_host_listings_count_entire_homes        24
calculated_host_listings_count_private_rooms       24
calculated_host_listings_count_shared_rooms        21
reviews_per_month                               24131
Length: 79, dtype: int64

In [4]:
clean_df = df.copy()

'''
We'll start with missing values and figure out how to handle each column, 
and determing whether it needs to be dropped or not.
'''

# since we have over 96000 records, and only 18 ids missing (Less than 0001%).
# we'll drop the the records with missing ids. ID's should not be fabricated

clean_df = clean_df.dropna(subset=["id"])

# clean_df.isna().sum()

# then we check to ensure all columns are properly converted

clean_df = df.drop(columns=["scrape_id", "source", "estimated_revenue_l365d", "first_review", "last_review"])

def convert_to_numeric(df, wanted_ratio):
    for col in df.columns:
        # first i convert to see how many values were successfully converted
        converted = pd.to_numeric(df[col], errors="coerce")
        
        # here we calculate how many values successfully converted
        convertible_ratio = converted.notna().mean()

        # here we use the wanted to ratio to determine whether you want to convert the column
        if convertible_ratio > wanted_ratio:
            # print(f"Converting '{col}' to numeric ({convertible_ratio*100:.1f}% convertible)")
            df[col] = converted   # Safe to convert
        # else:
            # print(f"NOT converting '{col}' ({convertible_ratio*100:.1f}% convertible)")

    
    return df

def convert_to_string(df, wanted_ratio):
    for col in df.columns:
        # Try numeric conversion temporarily
        converted = pd.to_numeric(df[col], errors="coerce")
        
        # If value is NaN in numeric conversion → it's a non-numeric value
        non_numeric_ratio = converted.isna().mean()
        # print(f"'{col}': {non_numeric_ratio*100:.1f}% string")

        # If mostly non-numeric → convert column to string
        if non_numeric_ratio > wanted_ratio:
            # print(f"  → Converting '{col}' to string")
            df[col] = df[col].astype("string")
        # else:
        #     print(f"  → Keeping '{col}' as is")
        
    return df

# the reason we are converting to Int64 instead of int64, is because some values are missing
# this integer type supports pd.NA
# regular int64 doesnt support this
def convert_percentages_to_int(df, columns):
    for col in columns:
        df[col] = df[col].str.replace("%", "", regex=False)
        df[col] = pd.to_numeric(df[col], errors="coerce").astype("Int64")

    return df

def convert_floats_to_int(df):
    for col in df.columns:
        if df[col].dtype == "float64":
            missing_val = df[col].dropna()

            if (df[col].dropna() % 1 == 0).all() and len(missing_val) > 0:
                df[col] = df[col].astype("Int64")

    return df 


clean_df = convert_to_numeric(clean_df, 0.5)
clean_df = convert_to_string(clean_df, 0.95)
clean_df = convert_percentages_to_int(clean_df, ["host_response_rate", "host_acceptance_rate"])
clean_df = convert_floats_to_int(clean_df)

# clean_df
# clean_df.dtypes

# non_missing = clean_df["host_id"].dropna()
# bad_host_ids = non_missing[non_missing % 1 != 0]

# bad_host_ids.head(20), bad_host_ids.shape

# i want to convert clearly int columns to integer values. 
# however some of the cells are corrupted and need to be dropped
# before that we need to see which ones have been corrupted

problem_columns = {}
def check_corrupted_float_columns(df):
    
    for col in df.columns:
        if df[col].dtype == "float64":
            non_missing = df[col].dropna()
            bad_values = non_missing[non_missing % 1 != 0]

            if len(bad_values) > 0:
                problem_columns[col] = len(bad_values)

    return problem_columns